Modalités pédagogiques :
Pratique 1 (travail est individuel - livrable -> notebook):

Installez MySQL ou PostgreSQL sur votre machine
Faites vos premières requêtes SQL dans votre script Python et remplissez votre base de données avec les données des films
Vos requêtes doivent couvrir la totalité des principes CRUD -> expliquez ces principes
Créez plusieurs tables et expérimentez les jointures
Faites un panorama de ce que vous êtes parvenu à faire sur un notebook commenté et propre.
​

Pratique 2 (travail individuel ou en groupe, organisation libre - livrable -> slides) :

Faites un panorama des types de données en SQL (et entre SGBD, MySQL et consorts)
Faites des recherches sur le modèle conceptuel de données
Réfléchissez à comment vous pourriez organiser vos tables dans votre base de données
Prenez en compte le fait que votre base de données doit pour être maintenable et extensible
Faites des recherches sur le diagramme de classes
Faites des slides pour regrouper et synthétiser ce que vous avez compris
​

Bonus : Faites des recherches sur les domaines suivants (livrable -> Notebook ou slides):

Le principe ACID (Atomicité, Cohérence, Isolation, Durabilité)
Les bases de données NoSQL (ou autre solution plus exotique, vector database etc...) -> Faites un compte rendu de vos recherches / faites des tests en Python/CLI/autre techno

# Mettre en place la base de données

## Import des librairies

In [1]:
import pandas as pd
import mysql.connector
from mysql.connector import Error

## Connexion à la base de données

In [2]:
# Fonction pour établir la connexion à la base de données
def create_connection(host, user, password, database):
    try:
        connection = mysql.connector.connect(
            host=host,
            user=user,
            password=password,
            database=database  # Ajoutez cette ligne pour spécifier la base de données
        )
        if connection.is_connected():
            print(f"Connected to MySQL Server: {host} with database: {database}")
            return connection

    except Error as e:
        print(f"Error: {e}")
        return None

## Executer des requêtes SQL

In [3]:
# Fonction pour exécuter une requête SQL
def execute_query(connection, query):
    try:
        cursor = connection.cursor()
        cursor.execute(query)
        connection.commit()
        print("Query executed successfully")
    except Error as e:
        print(f"Error: {e}")

## Créer une base de données

In [4]:
# Fonction pour créer une base de données
def create_database(connection, database_name):
    create_database_query = f"CREATE DATABASE IF NOT EXISTS {database_name}"
    execute_query(connection, create_database_query)

## Créer une table à partir d'un dataframe

In [5]:
# Fonction pour créer une table à partir d'un DataFrame
def create_table_from_dataframe(connection, dataframe, table_name):
    # Créer une table avec les colonnes correspondantes au DataFrame
    columns = ', '.join([f"{col} VARCHAR(255)" for col in dataframe.columns])
    create_table_query = f"CREATE TABLE {table_name} ({columns})"
    execute_query(connection, create_table_query)

In [6]:
# Fonction pour créer une table "users"
def create_users_table(connection):
    create_users_table_query = """
    CREATE TABLE users (
        id INT AUTO_INCREMENT PRIMARY KEY,
        username VARCHAR(50) NOT NULL UNIQUE,
        email VARCHAR(100) NOT NULL UNIQUE,
        password_hash VARCHAR(255) NOT NULL
    )
    """
    execute_query(connection, create_users_table_query)

## Charger les données dans une table depuis un fichier CSV

In [7]:
# Fonction pour charger les données depuis un fichier CSV dans une table MySQL
def load_data_into_mysql(connection, dataframe, table_name):
    for _, row in dataframe.iterrows():
        values = ', '.join([f"%s" for _ in row])  # Utilisation de paramètres de substitution (%s)
        insert_query = f"INSERT INTO {table_name} VALUES ({values})"
        cursor = connection.cursor()
        cursor.execute(insert_query, tuple(row))  # Utilisation de tuple(row) pour fournir les valeurs
        connection.commit()

## Parametres de connexion à la base de données

In [8]:
# Paramètres de connexion à la base de données
host = "localhost"
user = "root"
password = "toor"
database = "films_database"
table_name = "films"

## Utilisation des fonctions

In [9]:
# Créer la connexion à la base de données
connection = create_connection(host, user, password, database)

Connected to MySQL Server: localhost with database: films_database


In [10]:
# Créer la base de données
create_database(connection, database)

Query executed successfully


In [11]:
# Fermer la connexion à la base de données non spécifiée
connection.close()

In [12]:
# Rétablir la connexion à la base de données spécifiée
connection = create_connection(host, user, password, database)

Connected to MySQL Server: localhost with database: films_database


In [13]:
# Charger les données depuis le fichier CSV
movies_data = pd.read_csv('movies.csv')

In [14]:
# Créer la table à partir du DataFrame
create_table_from_dataframe(connection, movies_data, table_name)

Query executed successfully


In [15]:
# Créer la table "users"
create_users_table(connection)

Query executed successfully


In [16]:
# Tronquer les valeurs de la colonne 'anecdote' à une longueur spécifique
max_length = 255  # Remplacez par la longueur maximale de votre colonne
truncated_anecdotes = movies_data['anecdote'].apply(lambda x: x[:max_length] if isinstance(x, str) else x)
movies_data['anecdote'] = truncated_anecdotes
# Charger les données dans la table MySQL
load_data_into_mysql(connection, movies_data, table_name)

In [17]:
# Fermer la connexion
connection.close()

# CRUD (Create, Read, Update, Delete)

In [18]:
# Créer la connexion à la base de données
connection = create_connection(host, user, password, database)

Connected to MySQL Server: localhost with database: films_database


## Create

In [19]:
# Fonction pour effectuer une opération de création (Create)
def create_record(connection, table_name, values):
    values_str = ', '.join([f"'{str(value)}'" for value in values])
    insert_query = f"INSERT INTO {table_name} VALUES ({values_str})"
    execute_query(connection, insert_query)

## Read

In [20]:
# Fonction pour effectuer une opération de lecture (Read)
def read_records(connection, table_name):
    select_query = f"SELECT * FROM {table_name}"
    cursor = connection.cursor(dictionary=True)
    cursor.execute(select_query)
    records = cursor.fetchall()
    return records

## Update

In [21]:
# Fonction pour effectuer une opération de mise à jour (Update)
def update_record(connection, table_name, column, new_value, condition_column, condition_value):
    update_query = f"UPDATE {table_name} SET {column} = '{new_value}' WHERE {condition_column} = '{condition_value}'"
    execute_query(connection, update_query)

## Delete

In [22]:
# Fonction pour effectuer une opération de suppression (Delete)
def delete_record(connection, table_name, condition_column, condition_value):
    delete_query = f"DELETE FROM {table_name} WHERE {condition_column} = '{condition_value}'"
    execute_query(connection, delete_query)

## Exemple utilisation

In [23]:
# Create
create_record(connection, table_name, ["New Movie", "2023", "Action"])

Error: 1136 (21S01): Column count doesn't match value count at row 1


In [24]:
# Read
records = read_records(connection, table_name)
print("Records after creation:")
print(records)

Records after creation:


IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [25]:
# Update
update_record(connection, table_name, "genre", "Sci-Fi", "title", "New Movie")

Error: 1054 (42S22): Unknown column 'title' in 'where clause'


In [26]:
# Read after update
records_after_update = read_records(connection, table_name)
print("Records after update:")
print(records_after_update)

Records after update:


IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [27]:
# Delete
delete_record(connection, table_name, "title", "New Movie")

Error: 1054 (42S22): Unknown column 'title' in 'where clause'


In [28]:
# Read after delete
records_after_delete = read_records(connection, table_name)
print("Records after delete:")
print(records_after_delete)

Records after delete:

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [29]:
# Fermer la connexion
connection.close()